[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ourownstory/neural_prophet/blob/master/example_notebooks/autoregression_yosemite_temps.ipynb)

# Hyperparameter optimization with Ray Tune

We introduce the module for hyperparameter otpimization with Ray Tune.

It supports automatic tuning, with predefined by us hyperparameter sets, as well as manual tuning with user-provided configuration of the parameters.

Firstly, we will show how it works with NP model in automated mode.

In [1]:
# install NeuralProphet from our repository
# !pip install git+https://github.com/adasegroup/neural_prophet.git # may take a while
# !pip install tensorboardX

In [2]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet

In [3]:
from neuralprophet.hyperparameter_tuner import tune_hyperparameters

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)



In [4]:
if 'google.colab' in str(get_ipython()):
    data_location = "https://raw.githubusercontent.com/adasegroup/neural_prophet/master/"
else:
    data_location = "../"

df = pd.read_csv(data_location + "example_data/yosemite_temps.csv")
df.head(3)

,ds,y
0,2017-05-01 00:00:00,27.8
1,2017-05-01 00:05:00,27.0
2,2017-05-01 00:10:00,26.8


In [8]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP',
                                               df,
                                               freq)

In [5]:
best_params

{'growth': 'linear',
 'n_changepoints': 10,
 'changepoints_range': 0.5,
 'trend_reg': 10.0,
 'yearly_seasonality': False,
 'weekly_seasonality': True,
 'daily_seasonality': True,
 'seasonality_mode': 'additive',
 'seasonality_reg': 0.0,
 'n_lags': 30,
 'd_hidden': 8,
 'num_hidden_layers': 2,
 'ar_sparsity': 0.8,
 'learning_rate': 0.024150905458487568,
 'loss_func': 'Huber',
 'normalize': 'auto'}

This dictionary can further be used in initialization of NeuralProphet model.

This function has also additional parameters:
- **num_epochs**: Max possible number of epochs to train each model.
- **num_samples**: Number of samples from hyperparameter spaces to check.
- **resources_per_trial**: Resources per trial setting for ray.tune.run, {'cpu': 1, 'gpu': 2} for example



This function additionally will output the dataframe with detailed result of each trial if return_results is set to True.

In [7]:
results_df[['config.growth', 'config.n_changepoints', 'config.changepoints_range',
       'config.trend_reg', 'config.yearly_seasonality',
       'config.weekly_seasonality', 'config.daily_seasonality',
       'config.seasonality_mode', 'config.seasonality_reg', 'config.n_lags',
       'config.d_hidden', 'config.num_hidden_layers', 'config.ar_sparsity',
       'config.learning_rate', 'config.loss_func', 'config.normalize']].head()

,config.growth,config.n_changepoints,config.changepoints_range,config.trend_reg,config.yearly_seasonality,config.weekly_seasonality,config.daily_seasonality,config.seasonality_mode,config.seasonality_reg,config.n_lags,config.d_hidden,config.num_hidden_layers,config.ar_sparsity,config.learning_rate,config.loss_func,config.normalize
trial_id,,,,,,,,,,,,,,,,
8f12d_00000,off,10,0.9,10.0,False,False,True,multiplicative,1.0,100,64,8,0.8,0.000733,MSE,off
8f12d_00001,linear,100,0.8,0.5,True,False,True,additive,0.0,10,128,16,0.8,0.019590,MSE,soft
8f12d_00002,linear,10,0.5,0.0,True,False,False,multiplicative,0.0,30,128,2,0.1,0.000306,MSE,standardize
8f12d_00003,linear,10,0.5,10.0,True,True,True,multiplicative,0.5,30,8,8,0.8,0.000195,Huber,off
8f12d_00004,linear,10,0.5,0.5,True,True,True,additive,0.5,10,64,2,0.8,0.008229,MSE,soft


## Manual mode

In case of manual mode, a user must provide a config dictionary with hyperparameter spaces in compatability with Ray Tune api.

We provide a minimal example below, for more information on Search Spaces withit this link https://docs.ray.io/en/master/tune/api_docs/search_space.html?highlight=tune.choice

In [9]:
from ray import tune

config = {'n_lags': tune.grid_search([10, 20, 30]),
          'learning_rate': tune.loguniform(1e-4, 1e-1),
          'num_hidden_layers': tune.choice([2, 8, 16])}

In [ ]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP', 
                                               df,
                                               freq, 
                                               mode = 'manual',
                                              config = config)

(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable 

(pid=39357) 
(pid=39357)   | Name          | Type          | Params
(pid=39357) ------------------------------------------------
(pid=39357) 0 | season_params | ParameterDict | 18    
(pid=39357) 1 | ar_net        | ModuleList    | 3.7 K 
(pid=39357) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=39357) ------------------------------------------------
(pid=39357) 3.7 K     Trainable params
(pid=39357) 0         Non-trainable params
(pid=39357) 3.7 K     Total params
(pid=39357) 0.015     Total estimated model params size (MB)
(pid=39357) WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(pid=39357)   warnings.warn(*args, **

  0%|          | 0/100 [00:00<?, ?it/s]
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=41823) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=41823) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=41823) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]
(pid=41823) GPU available: False, used: False
(pid=41823) TPU available: False, using: 0 TPU cores
(pid=41823) GPU available: False, used: False
(pid=41823) TPU available: False, using: 0 TPU cores
  0%|        

  0%|          | 0/100 [00:00<?, ?it/s]
(pid=41845) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=41845) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=41845) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=41845) TPU available: False, using: 0 TPU cores
(pid=41845) GPU available: False, used: False
(pid=41845) TPU available: False, using: 0 TPU cores
(pid=41845) 
(pid=41845)   | Name          | Type          | Params
(pid=41845) ------------------------------------------------
(pid=41845) 0 | season_params | ParameterDict | 18    
(pid=41845) 1 | ar_net        | ModuleList    | 2.0 K 
(pid=41845) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=41845) ------------------------------------------------
(pid=41845) 2.0 K     Trainable params
(pid=418

(pid=41902) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=41902) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=41902) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=41902) TPU available: False, using: 0 TPU cores
(pid=41902) GPU available: False, used: False
(pid=41902) TPU available: False, using: 0 TPU cores
  0%|          | 0/100 [00:00<?, ?it/s]
(pid=41902) 
(pid=41902)   | Name          | Type          | Params
(pid=41902) ------------------------------------------------
(pid=41902) 0 | season_params | ParameterDict | 18    
(pid=41902) 1 | ar_net        | ModuleList    | 15.9 K
(pid=41902) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=41902) ------------------------------------------------
(pid=41902) 15.9 K    Trainable params
(pid=419

(pid=42006) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=42006) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=42006) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=42006) TPU available: False, using: 0 TPU cores
(pid=42006) GPU available: False, used: False
(pid=42006) TPU available: False, using: 0 TPU cores
(pid=42006) 
(pid=42006)   | Name          | Type          | Params
(pid=42006) ------------------------------------------------
(pid=42006) 0 | season_params | ParameterDict | 18    
(pid=42006) 1 | ar_net        | ModuleList    | 2.0 K 
(pid=42006) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=42006) ------------------------------------------------
(pid=42006) 2.0 K     Trainable params
(pid=42006) 0         Non-trainable params
(pid=

  0%|          | 0/100 [00:00<?, ?it/s]
(pid=42045) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=42045) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=42045) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=42045) TPU available: False, using: 0 TPU cores
(pid=42045) GPU available: False, used: False
(pid=42045) TPU available: False, using: 0 TPU cores
  0%|          | 0/100 [00:00<?, ?it/s]
(pid=42045) 
(pid=42045)   | Name          | Type          | Params
(pid=42045) ------------------------------------------------
(pid=42045) 0 | season_params | ParameterDict | 18    
(pid=42045) 1 | ar_net        | ModuleList    | 15.9 K
(pid=42045) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=42045) ------------------------------------------------
(pid=42

(pid=42109) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=42109) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=42109) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=42109) TPU available: False, using: 0 TPU cores
(pid=42109) GPU available: False, used: False
(pid=42109) TPU available: False, using: 0 TPU cores
(pid=42109) 
(pid=42109)   | Name          | Type          | Params
(pid=42109) ------------------------------------------------
(pid=42109) 0 | season_params | ParameterDict | 18    
(pid=42109) 1 | ar_net        | ModuleList    | 7.9 K 
(pid=42109) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=42109) ------------------------------------------------
(pid=42109) 8.0 K     Trainable params
(pid=42109) 0         Non-trainable params
(pid=

(pid=42209) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=42209) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=42209) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=42209) TPU available: False, using: 0 TPU cores
(pid=42209) GPU available: False, used: False
(pid=42209) TPU available: False, using: 0 TPU cores
(pid=42209) 
(pid=42209)   | Name          | Type          | Params
(pid=42209) ------------------------------------------------
(pid=42209) 0 | season_params | ParameterDict | 18    
(pid=42209) 1 | ar_net        | ModuleList    | 2.0 K 
(pid=42209) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=42209) ------------------------------------------------
(pid=42209) 2.0 K     Trainable params
(pid=42209) 0         Non-trainable params
(pid=

(pid=42508) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=42508) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=42508) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=42508) TPU available: False, using: 0 TPU cores
(pid=42508) GPU available: False, used: False
(pid=42508) TPU available: False, using: 0 TPU cores
(pid=42508) 
(pid=42508)   | Name          | Type          | Params
(pid=42508) ------------------------------------------------
(pid=42508) 0 | season_params | ParameterDict | 18    
(pid=42508) 1 | ar_net        | ModuleList    | 7.9 K 
(pid=42508) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=42508) ------------------------------------------------
(pid=42508) 8.0 K     Trainable params
(pid=42508) 0         Non-trainable params
(pid=

In [ ]:
results_df

In [ ]:
best_params

# Other models

Hyperparameter optimization can be also run for each of 4 models, available in Neuralprophet library: LSTM, NBeats, TFT and DeepAR. In order to run optimization for particular model, the procedure is the same, you just need to insert the name of preferable model.

An example for LSTM can be found below:

In [8]:
freq = '5min'
best_params, results_df = tune_hyperparameters('LSTM',
                                               df,
                                               freq)

(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable 

  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=28974) TPU available: False, using: 0 TPU cores
(pid=28974) GPU available: False, used: False
(pid=28974) TPU available: False, using: 0 TPU cores
(pid=28974) 
(pid=28974)   | Name      | Type         | Params
(pid=28974) -------------------------------------------
(pid=28974) 0 | lstm      | LSTM         | 30.0 K
(pid=28974) 1 | linear    | Linear       | 17    
(pid=28974) 2 | loss_func | SmoothL1Loss | 0     
(pid=28974) -------------------------------------------
(pid=28974) 30.0 K    Trainable params
(pid=28974) 0         Non-trainable params
(pid=28974) 30.0 K    Total params
(pid=28974) 0.120     Total estimated model params size (MB)
(pid=28974) WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Cons

(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=29042) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=29042) TPU available: False, using: 0 TPU cores
(pid=29042) GPU available: False, used: False
(pid=29042) TPU available: False, using: 0 TPU cores
(pid=29042) 
(pid=29042)   | Name      | Type         | Params
(pid=29042) -------------------------------------------
(pid=29042) 0 | lstm      | LSTM         | 275 K 
(pid=29042) 1 | linear    | Linear       | 65    
(pid=29042) 2 | loss_func | SmoothL1Loss | 0     
(pid=29042) --------------------------------------

(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=29108) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=29108) TPU available: False, using: 0 TPU cores
(pid=29108) GPU available: False, used: False
(pid=29108) TPU available: False, using: 0 TPU cores
(pid=29108) 
(pid=29108)   | Name      | Type         | Params
(pid=29108) -------------------------------------------
(pid=29108) 0 | lstm      | LSTM         | 6.2 M 
(pid=29108) 1 | linear    | Linear       | 257   
(pid=29108) 2 | loss_func | SmoothL1Loss | 0     
(pid=29108) --------------------------------------

2021-05-21 14:48:23,922	ERROR tune.py:545 -- Trials did not complete: [train_LSTM_tune_ddb98_00000, train_LSTM_tune_ddb98_00004, train_LSTM_tune_ddb98_00009, train_LSTM_tune_ddb98_00011, train_LSTM_tune_ddb98_00013, train_LSTM_tune_ddb98_00014, train_LSTM_tune_ddb98_00015, train_LSTM_tune_ddb98_00016, train_LSTM_tune_ddb98_00017, train_LSTM_tune_ddb98_00018, train_LSTM_tune_ddb98_00019, train_LSTM_tune_ddb98_00020, train_LSTM_tune_ddb98_00021, train_LSTM_tune_ddb98_00022, train_LSTM_tune_ddb98_00023, train_LSTM_tune_ddb98_00024, train_LSTM_tune_ddb98_00025, train_LSTM_tune_ddb98_00026, train_LSTM_tune_ddb98_00027, train_LSTM_tune_ddb98_00028, train_LSTM_tune_ddb98_00029, train_LSTM_tune_ddb98_00030, train_LSTM_tune_ddb98_00031, train_LSTM_tune_ddb98_00032, train_LSTM_tune_ddb98_00033, train_LSTM_tune_ddb98_00034, train_LSTM_tune_ddb98_00035, train_LSTM_tune_ddb98_00036, train_LSTM_tune_ddb98_00037, train_LSTM_tune_ddb98_00038, train_LSTM_tune_ddb98_00039]
2021-05-21 14:48:23,923	WARNIN

In [9]:
best_params

{'learning_rate': 0.013718466327407178,
 'd_hidden': 128,
 'n_lags': 100,
 'num_hidden_layers': 2,
 'lstm_bias': True,
 'lstm_bidirectional': False}

In [10]:
results_df.head()

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.learning_rate,config.d_hidden,config.n_lags,config.num_hidden_layers,config.lstm_bias,config.lstm_bidirectional
trial_id,,,,,,,,,,,,,,,,,,,,,
ddb98_00000,0.047840,184.533827,False,NaN,NaN,8.0,a26094effe6246d5af6b147ae60a5933,2021-05-21_14-47-36,1.621598e+09,1486.938134,...,1486.938134,0.0,8.0,"0_d_hidden=128,learning_rate=0.000337,lstm_bia...",0.000337,128.0,30.0,16.0,False,True
ddb98_00001,0.051339,48.592918,True,NaN,NaN,20.0,4ebbb108e09e4b2986c3835634ed0295,2021-05-21_14-38-22,1.621597e+09,932.583967,...,932.583967,0.0,20.0,"1_d_hidden=8,learning_rate=0.0097144,lstm_bias...",0.009714,8.0,100.0,8.0,False,True
ddb98_00002,0.255517,28.091819,True,NaN,NaN,10.0,4be3033d37004d4f98c47a2d0d564c11,2021-05-21_14-27-36,1.621596e+09,285.940451,...,285.940451,0.0,10.0,"2_d_hidden=64,learning_rate=0.00010157,lstm_bi...",0.000102,64.0,100.0,8.0,False,False
ddb98_00003,0.000029,11.933487,True,NaN,NaN,100.0,44266641f7124ffda7d62dbdab2b0b49,2021-05-21_14-43-17,1.621597e+09,1227.605633,...,1227.605633,0.0,100.0,"3_d_hidden=128,learning_rate=0.013718,lstm_bia...",0.013718,128.0,100.0,2.0,True,False
ddb98_00004,0.047503,91.865763,False,NaN,NaN,16.0,f6f09c2bc638437498bd2d01fd44961d,2021-05-21_14-47-51,1.621598e+09,1501.235601,...,1501.235601,0.0,16.0,"4_d_hidden=8,learning_rate=0.0013775,lstm_bias...",0.001378,8.0,100.0,16.0,False,True


In [5]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NBeats',
                                               df,
                                               freq, 
                                               num_epochs = 5,
                                              num_samples = 3)

2021-05-21 22:44:37,062	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-21 22:44:38,947	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(pid=39358) GPU available: False, used: False
(pid=39358) TPU available: False, using: 0 TPU cores
(pid=39362) GPU available: False, used: False
(pid=39362) TPU available: False, using: 0 TPU cores
(pid=39359) GPU available: False, used: False
(pid=39359) TPU available: False, using: 0 TPU cores
(pid=39362) GPU available: False, used: False
(pid=39362) TPU available: False, using: 0 TPU cores
(pid=39359) GPU available: False, used: False
(pid=39359) TPU available: False, using: 0 TPU cores
(pid=39358) GPU available: False, used: False
(pid=39358) TPU available: False, using: 0 TPU cores
(pid=39362) 
(pid=39362)   | Nam

(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 

(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 

(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20])
(pid=39358)  
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(p

(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([13, 

(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([13, 20]) torch.Size([13, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39362) torch.Size([64, 

(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 

(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39358) torch.Size([64, 20]) torch.Size([64, 20])
(pid=39359) torch.Size([64, 

In [6]:
best_params

{'learning_rate': 0.0007916629501075156, 'context_length': 100}

In [7]:
results_df

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.learning_rate,config.context_length
trial_id,,,,,,,,,,,,,,,,,,,
faaa5_00000,3.072153,18.586122,True,None,None,5,7b67c8b9c6eb44aca2845bc8e56a4ebb,2021-05-21_22-46-19,1621626379,97.502764,39359,MacBook-Pro-Polina.local,192.168.0.7,97.502764,0,5,"0_context_length=100,learning_rate=0.00015071",0.000151,100
faaa5_00001,3.030095,17.945912,True,None,None,5,1656dc1c435f4cc883e27b4c35c1d52a,2021-05-21_22-46-14,1621626374,92.765030,39362,MacBook-Pro-Polina.local,192.168.0.7,92.765030,0,5,"1_context_length=30,learning_rate=0.0029214",0.002921,30
faaa5_00002,2.599046,18.865901,True,None,None,5,3fb214eaabee41b5b0fba90891813774,2021-05-21_22-46-19,1621626379,97.607086,39358,MacBook-Pro-Polina.local,192.168.0.7,97.607086,0,5,"2_context_length=100,learning_rate=0.00079166",0.000792,100


In [5]:
np.min(10, 2)

AxisError: axis 2 is out of bounds for array of dimension 0

In [7]:
best_params

{'learning_rate': 0.0601654074086814,
 'context_length': 10,
 'hidden_size': 16,
 'attention_head_size': 1}

In [8]:
results_df

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.learning_rate,config.context_length,config.hidden_size,config.attention_head_size
trial_id,,,,,,,,,,,,,,,,,,,,,
217f9_00000,0.585543,12.794128,True,None,None,10,374ac13a0d874cd78833bb53a8fd825b,2021-05-21_15-24-11,1621599851,992.677625,...,MacBook-Pro-Polina.local,192.168.0.7,992.677625,0,10,"0_attention_head_size=2,context_length=10,hidd...",0.000435,10,16,2
217f9_00001,0.335199,11.922237,True,None,None,100,91d1cf0e9d574b45b972a8df7e14a0bc,2021-05-21_15-39-29,1621600769,1911.347692,...,MacBook-Pro-Polina.local,192.168.0.7,1911.347692,0,100,"1_attention_head_size=1,context_length=10,hidd...",0.060165,10,16,1
217f9_00002,0.643699,22.244758,True,None,None,10,e9709940e5de48da840b8de46ebc4544,2021-05-21_15-22-29,1621599749,890.869505,...,MacBook-Pro-Polina.local,192.168.0.7,890.869505,0,10,"2_attention_head_size=2,context_length=10,hidd...",0.031605,10,32,2


In [ ]:
best_params

In [ ]:
results_df